# PyTorch_Tutorial_3 : NEURAL NETWORKS

### 이 노트는 아래의 웹사이트의 코드를 따라가며 해설하였다.

https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html#sphx-glr-beginner-blitz-neural-networks-tutorial-py

## Neural Network with PyTorch

- Neural networks는 torch.nn 패키지를 이용해 만들 수 있다.
- nn 패키지는 autograd 패키지의 기능을 이용해 모델을 정의하고 미분한다.
- nn.Module클래스에서 layers를 정의하고 모델의 결과값을 반환하는 forward 메소드를 제공한다.

#### NOTE ) Neural Network의 학습순서는 위 사이트에 다음과 같이 언급되어 있으니 참고할 것! 

- Define the neural network that has some learnable parameters (or weights)
- Iterate over a dataset of inputs
- Process input through the network
- Compute the loss (how far is the output from being correct)
- Propagate gradients back into the network’s parameters
- Update the weights of the network, typically using a simple update rule: weight = weight - learning_rate * gradient

In [1]:
from pprint import pprint
import torch
import torch.nn as nn
import torch.nn.functional as F

### 1. Defining Model

#### NOTE ) 아래 모델의 시각화는 위 사이트를 참조할 것! 

In [2]:
# 모델 정의 시, Module클래스를 상속할 것!
# Expected input size of this net (LeNet) is 32x32
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b (linear op.인 Wx에 상수를 더해서 Wx+b를 계산하는 것을 affine op.라 한다.)
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    # 이부분은 항상 사용자가 정의해야한다.
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number.
        # 즉, (2,2) 대신에 2라고만 써도 된다.
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    # 한번 학습할때마다 x에는 minibatch size만큼의 데이터가 들어있다.
    # 따라서 tensor size에서 첫번째 요소(minibatch size)를 제외하고,
    # "하나의 데이터"에 들어있는 전체 요소의 갯수를 구하는 것이다.
    # torch.nn only supports mini-batches.
    # The entire torch.nn package only supports inputs that are a mini-batch of samples,
    # and not a single sample.
    # That's why nn.Conv2d will take in a 4D Tensor of nSamples x nChannels x Height x Width
    # If you have a single sample, just use input.unsqueeze(0) to add a fake batch dimension.
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

# 위에 정의한 모델의 인스턴스 생성
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [3]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight
print(params[1].size())  # conv1's .bias

10
torch.Size([6, 1, 3, 3])
torch.Size([6])


In [4]:
# Let's try a random 32x32 input
my_input = torch.randn(1, 1, 32, 32)
out = net(my_input)    # forward 메소드가 작동해서 input에 대한 현재모델의 예측값을 반환한다.
print(out)

tensor([[ 0.0436,  0.2149, -0.0191, -0.0741,  0.0753, -0.0155,  0.0328, -0.1276,
          0.1156, -0.0460]], grad_fn=<AddmmBackward>)


In [5]:
# Zero the gradient buffers of all parameters and backprops with random gradients:
net.zero_grad()
out.backward(torch.randn(1, 10))    # out이 스칼라가 아니므로, 이와 같은 사이즈의 임의의 텐서를 argument에 대입!

### 2. Loss Function

In [6]:
output = net(my_input)
target = torch.randn(10)     # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(1.4170, grad_fn=<MseLossBackward>)


In [7]:
# next_functions 은 연산을 거슬러 올라간다.
print(loss.grad_fn)  # MSELoss
print('')
print(loss.grad_fn.next_functions)  # Linear
print('')
pprint(loss.grad_fn.next_functions[0][0].next_functions)  # Relu
print('')
print(loss.grad_fn.next_functions[0][0].next_functions[0][0].next_functions)
print(loss.grad_fn.next_functions[0][0].next_functions[1][0].next_functions)
print(loss.grad_fn.next_functions[0][0].next_functions[2][0].next_functions)


((<AddmmBackward object at 0x000001EC1846E208>, 0),)

((<AccumulateGrad object at 0x000001EC1846E208>, 0),
 (<ReluBackward0 object at 0x000001EC1846E308>, 0),
 (<TBackward object at 0x000001EC1846EDC8>, 0))

()
((<AddmmBackward object at 0x000001EC1846E3C8>, 0),)
((<AccumulateGrad object at 0x000001EC1846E208>, 0),)


- The neural network package contains various modules and loss functions that form the building blocks of deep neural networks.
- A full list with documentation is here: https://pytorch.org/docs/stable/nn.html.

### 3. Backprop

In [8]:
# Let's have a look at conv1’s bias gradients before and after the backward.

# zeroes the gradient buffers of all parameters,
# else gradients will be accumulated to existing gradients.
net.zero_grad()

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0153,  0.0124, -0.0236, -0.0029, -0.0088,  0.0003])


### 4. Update the weights

- The simplest update rule used in practice is the Stochastic Gradient Descent (SGD): weight = weight - learning_rate * gradient
- SGD, Nesterov-SGD, Adam, RMSProp 등 다양한 update 방법들을 간편하게 사용하기 위해 PyTorch에서는 torch.optim 패키지를 제공한다.

In [9]:
# Bias of conv1 before updates
print(list(net.parameters())[1])

Parameter containing:
tensor([ 0.2367,  0.0320, -0.0896, -0.2898,  0.2047,  0.0322],
       requires_grad=True)


In [10]:
learning_rate = 0.1
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [11]:
# Bias of conv1 after updates
print(list(net.parameters())[1])

Parameter containing:
tensor([ 0.2383,  0.0308, -0.0872, -0.2895,  0.2056,  0.0322],
       requires_grad=True)


In [12]:
# Update process above can be implemented in short length by torch.optim package
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zeros the gradient buffers (instead of net.zero_grad())
output = net(my_input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update